# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global data
    ma_len = len(y_axis)
    
    x_axis = []
    for i in range(ma_len):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_ma(ma_list):
    global data
    ma_len = len(ma_list)
    sema_val = list(pd.DataFrame(ma_list).ewm(span=ma_len).mean()[0])[ma_len - 1]    
    return(sema_val)

## File paths

In [6]:
year = 2019
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [7]:
%%time
#df = pd.read_csv(source_file_path, nrows=1000000)
df = pd.read_csv(source_file_path)
df.head()

Wall time: 21.1 s


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [8]:
%%time
data = {}
data['number_of_ticks'] = 60
data['pip_diff'] = 0.00001
data['rsi_window'] = 14
data['sma_len'] = 60
data['lma_len'] = 120


df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|████████████████████████████████████████████████████████████████████████| 486439/486439 [02:00<00:00, 4046.05it/s]


Records : 486439
Wall time: 2min 5s


,tick_avg,spread_avg,tick_sd
0,1.146434,0.000582,0.000107
1,1.146692,0.000256,0.000031
2,1.146543,0.000482,0.000135
3,1.146387,0.000614,0.000055
4,1.146308,0.000470,0.000074


In [9]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 392 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_ma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_ma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

486380it [07:53, 1026.79it/s]
486320it [07:53, 1027.64it/s]

Wall time: 15min 46s


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope)

486321it [02:28, 3272.96it/s]
486261it [02:32, 3186.70it/s]

Wall time: 5min 1s


In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 486261
Wall time: 416 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
486256,1.121369,0.000070,0.000031,1.121619,1.121886,-0.000012,-0.000012,-0.000012,0.000002,0.000031,...,-0.000010,-0.000402,1.122104,1.121369,0.000735,0.000000,-54.732055,-42.921377,-0.000267,decrease
486257,1.121349,0.000063,0.000035,1.121607,1.121874,-0.000012,-0.000012,-0.000020,0.000002,0.000025,...,-0.000010,-0.000404,1.122104,1.121349,0.000755,0.000000,-54.551258,-43.341746,-0.000267,decrease
486258,1.121272,0.000059,0.000034,1.121592,1.121860,-0.000015,-0.000013,-0.000077,0.000002,0.000021,...,-0.000011,-0.000407,1.122082,1.121272,0.000811,0.000000,-54.371573,-43.759290,-0.000269,decrease
486259,1.121165,0.000058,0.000043,1.121573,1.121845,-0.000018,-0.000015,-0.000107,0.000002,0.000027,...,-0.000011,-0.000410,1.122082,1.121165,0.000917,0.000000,-54.210101,-44.180254,-0.000272,decrease
486260,1.121181,0.000347,0.000075,1.121556,1.121831,-0.000017,-0.000015,0.000016,0.000002,0.000027,...,-0.000012,-0.000412,1.122082,1.121165,0.000901,-0.000016,-54.065824,-44.599558,-0.000275,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 18 s


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same      322336   66.288680
decrease   82600   16.986762
increase   81325   16.724557


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
0,1.145336,0.000043,0.000015,1.145431,1.145711,-0.000008,-0.000010,0.000009,0.000021,0.000019,...,-0.000011,-0.000411,1.146194,1.145087,0.000858,-0.000248,-55.022721,-46.667969,-0.000280,same
1,1.145404,0.000050,0.000031,1.145426,1.145703,-0.000005,-0.000008,0.000068,0.000026,0.000017,...,-0.000010,-0.000414,1.146099,1.145087,0.000695,-0.000317,-54.960014,-47.114901,-0.000277,same
2,1.145440,0.000043,0.000005,1.145424,1.145696,-0.000003,-0.000007,0.000036,0.000027,0.000017,...,-0.000010,-0.000415,1.146099,1.145087,0.000658,-0.000353,-54.845264,-47.532376,-0.000272,same
3,1.145478,0.000044,0.000018,1.145422,1.145689,-0.000001,-0.000007,0.000037,0.000030,0.000016,...,-0.000010,-0.000415,1.146096,1.145087,0.000618,-0.000390,-54.671402,-47.920995,-0.000267,same
4,1.145457,0.000045,0.000025,1.145420,1.145682,-0.000002,-0.000007,-0.000021,0.000030,0.000010,...,-0.000010,-0.000416,1.146096,1.145087,0.000639,-0.000370,-54.440295,-48.281256,-0.000262,same
